# Update

Update the table and column comments on the database.

# Setup

Setup config, database, and libraries.

## Libraries

In [91]:
import geopandas as gpd
import os
import pandas as pd
import re
import yaml

from datetime import datetime
from edotenv import load_edotenv
from pandas.api.types import is_numeric_dtype
from pathlib import Path
from sqlalchemy import create_engine, text, inspect
from textwrap import dedent

## Config

Load settings from the `config.yml` file.

In [92]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
    
ver = config['Version']
abbrv = config['Abbreviation']

## Database Connection

To connect to the database, you need to ensure that you have saved your login with successfully with `bin/login.bat` or `bin/login.sh`:

In Windows, run:

```
bin\login
```

In Linux/Mac OS, run:

```
source bin/login.sh
```

If these commands run successfully, a `.env` file will be created.

In [93]:
load_edotenv('../.env')
engine = create_engine(os.environ['DB_URL'])
uengine = create_engine(os.environ['UPLOAD_DB_URL'])
is_om = os.environ['UPLOAD_DB_IS_OM'] == 'True'

**Note**: If this fails, try `bin/login.bat` or `bin/login.sh` again.

## Read Data Descriptions

Read data descriptions from the `src/config.yml` file using the `Description` keys under the `Data` list.

Also add stats to the data descriptions.

In [94]:
# Create the data descriptions from config.yml
ddescribe_config = []
for data in config['Data']:
    ddescribe_config.append({
        'dataset': data['Name'],
        'description': data['Description']
    })
ddescribe_config = pd.DataFrame(ddescribe_config)

# Read the known data descriptions
ddescribe = pd.read_csv(f'data/{abbrv}_data.csv')

# Update the known data descriptions
ddescribe = pd.merge(ddescribe[['dataset', 'columns', 'rows']], ddescribe_config, on = 'dataset', how = 'left')

# Sort by dataset name
ddescribe = ddescribe.sort_values(by = ['dataset'])

Preview data descriptions.

In [95]:
ddescribe

,dataset,columns,rows,description
0,healsl_rd1_adult,367,5003,Adult Verbal Autopsy (VA) data for survey roun...
1,healsl_rd1_adult_age,11,5003,Specific ages for the [Adult VA Data (HEAL-SL ...
2,healsl_rd1_adult_narrative,3,4987,Narratives for the [Adult VA Data (HEAL-SL Rou...
3,healsl_rd1_child,313,3004,Child Verbal Autopsy (VA) data for survey roun...
4,healsl_rd1_child_age,9,3004,Specific ages for the [Child VA Data (HEAL-SL ...
5,healsl_rd1_child_narrative,3,2998,Narratives for the [Child VA Data (HEAL-SL Rou...
6,healsl_rd1_geo,10,38527,Household Global Positioning System (GPS) loca...
7,healsl_rd1_household,61,344466,Household data for survey round 1 of the Healt...
8,healsl_rd1_household_age,3,343120,Specific household member ages for the [Househ...
9,healsl_rd1_neo,265,586,Neonate Verbal Autopsy (VA) data for survey ro...


## Read Data Dictionaries

Read data dictionary by getting the column comments from the database without the private or excluded columns.

In [96]:
ddict = []
for data in config['Data']:
    
    # Prepare query statement
    query = text(
        """
        SELECT
            table_name AS dataset,
            column_name AS column,
            data_type AS type,
            col_description((table_schema||'.'||table_name)::regclass::oid, ordinal_position) AS description
        FROM
            information_schema.columns
        WHERE
            table_name = :table
            AND
            table_schema = :schema
            AND
            NOT column_name = ANY(:cols)
        ORDER BY table_name, ordinal_position;
        """
    )
    
    # Get column comments for dataset
    comments = pd.read_sql(query, engine, params={
        'table': data['Table'],
        'schema': data.get('Schema', 'public'),
        'cols': config.get('Excluded Columns', [''])
    })
    
    # Modify dataset name
    comments['dataset'] = data['Name']
    
    # Change character to character varying for flexibility
    comments.loc[comments['type'] == 'character', ['type']] = 'character varying'
    
    # Add dataset ddict to appropriate round
    ddict.append(comments)

# Combine ddict
ddict = pd.concat(ddict, ignore_index=True)

Preview data dictionary.

In [97]:
ddict

,dataset,column,type,description
0,healsl_rd1_geo,areaid,bigint,Unique enumeration area identifier
1,healsl_rd1_geo,timelog,timestamp with time zone,Date and time of enumeration
2,healsl_rd1_geo,gps_latitude,text,GPS latitude of the house
3,healsl_rd1_geo,gps_latitude_alt,double precision,GPS latitude of the house (Data coordinator al...
4,healsl_rd1_geo,gps_longitude,text,GPS longitude of the house
...,...,...,...,...
2183,healsl_rd2_neo_age,age_unit_cod,text,How old was (s)he (the deceased)? (Originated ...
2184,healsl_rd2_neo_age,age_value_cod,integer,How old was (s)he (the deceased)? (Originated ...
2185,healsl_rd2_neo_narrative,rowid,bigint,Unique row identifier for this dataset
2186,healsl_rd2_neo_narrative,timelog,timestamp with time zone,Date and time of enumeration


# Database Update

Update comments and views in database with data dictionaries and descriptions.

## Upload Comments

Add table and column comments to uploaded dataset tables by:

1. Generating SQL for dataset table comment
2. Generating SQL for dataset column comments
3. Executing generated SQL statements above

In [98]:
sql = {}
for data in config['Data']:

    # Get info from config for dataset
    version = data.get('Version', ver)
    dataset = data['Name']
    table = f'{dataset}_v{version}'

    # Add schema if avail
    if 'Upload Schema' in data:
        schema = data['Upload Schema']
        table = f'{schema}.{table}'

    # Get ddict for dataset
    dd = ddict[ddict['dataset'] == dataset]

    # 1. Create sql for table comment
    squote = "'"
    dbquotes = "''"
    description = ddescribe[ddescribe['dataset'] == dataset]['description']
    description = description.tolist()[0].replace(squote, dbquotes)
    comment_query = f"COMMENT ON TABLE {table} IS '{description}';"

    # 2. Create sql for column comments
    ncols = dd.shape[0]
    col_query = [f"COMMENT ON COLUMN {table}.{r['column']} IS '{str(r['description']).replace(squote, dbquotes)}';" for i, r in dd.iterrows()]
    col_query = '\n'.join(col_query)

    # 3a. Add table and col comment statements
    query = f'--- {table} table comment\n' + comment_query \
        + f'\n\n--- {table} column comments (n={ncols})\n' + col_query
    sql[dataset] = query

# 3b. Combine and execute comment statements
comment_sql = '\n\n'.join(q for dataset, q in sql.items())
with uengine.connect() as connection:
    connection.execute(text(comment_sql))
    connection.execute(text('COMMIT;'))

## Upload Views

Create views for the uploaded tables with accompanying view/column comments.

In [99]:
sql = {}
for data in config['Data']:

    # Get info from config for dataset
    version = data.get('Version', ver)
    dataset = data['Name']
    table = f'{dataset}_v{version}'

    # Add schema if avail
    if 'Upload Schema' in data:
        schema = data['Upload Schema']
        table = f'{schema}.{table}'

    # Get ddict for dataset
    dd = ddict[ddict['dataset'] == dataset]

    # 1. Create view query
    view_query = f'DROP VIEW IF EXISTS {dataset}; CREATE OR REPLACE VIEW {dataset} AS (SELECT * FROM {table});'

    # 2. Create sql for view comment
    squote = "'"
    dbquotes = "''"
    description = ddescribe[ddescribe['dataset'] == dataset]['description']
    description = description.tolist()[0].replace(squote, dbquotes)
    cols = ddescribe[ddescribe['dataset'] == data['Name']]['columns'].tolist()[0]
    rows = ddescribe[ddescribe['dataset'] == data['Name']]['rows'].tolist()[0]
    comment_query = f"COMMENT ON VIEW {dataset} IS '{description}\n\n* **Columns**: {cols}\n\n* **Rows**: {rows}';"

    # 3. Create sql for column comments
    ncols = dd.shape[0]
    col_query = [f"COMMENT ON COLUMN {dataset}.{r['column']} IS '{str(r['description']).replace(squote, dbquotes)}';" for i, r in dd.iterrows()]
    col_query = '\n'.join(col_query)

    # 3a. Add view, comment, and column statements
    sql[dataset] = f'--- {dataset} view\n\n' + view_query \
        + f'\n\n--- {dataset} view comment\n\n' \
        + comment_query \
        + f'\n\n--- {dataset} view column comments (n={ncols})\n\n' \
        + col_query

# 3b. Combine and execute comment statements
view_sql = '\n\n'.join([q for dataset, q in sql.items()])
with uengine.connect() as connection:
    connection.execute(text(view_sql))
    connection.execute(text('COMMIT;'))

## Save Comments SQL

Save comments sql to file.

In [100]:
# Create folder to store database outputs
Path('database').mkdir(exist_ok=True)

# Save comments sql for tables
with open(f'database/{abbrv}_comments_v{ver}.sql', 'w') as file:
    file.write(comment_sql)

## Save Views SQL

Save views sql to file.

In [101]:
with open(f'database/{abbrv}_views_v{ver}.sql', 'w') as file:
    file.write(view_sql)

# Open Mortality Upload

Upload dataset and data records for https://openmortality.org

## Add Data Records

Query for the current data table or create one if it does not exist.

Then add a record for all the data in this dataset.

In [102]:
om_dtable = config['Data Table']
om_dschema = config.get('Data Schema', 'public')

with uengine.connect() as connection:
    
    # Create dataset table if not exists
    dcreate = text(dedent(
        f"""
        --- Data records table
        CREATE TABLE IF NOT EXISTS {om_dschema}.{om_dtable} (
            data_id SERIAL PRIMARY KEY,
            title VARCHAR,
            data_name VARCHAR UNIQUE,
            format VARCHAR,
            is_spatial BOOLEAN,
            permission VARCHAR,
            publish_date TIMESTAMP WITH TIME ZONE,
            last_updated_date TIMESTAMP WITH TIME ZONE,
            status VARCHAR,
            tag VARCHAR,
            data_desc VARCHAR,
            last_update_date VARCHAR,
            contact VARCHAR,
            category VARCHAR,
            data_columns BIGINT,
            data_rows BIGINT
        );
        """
    ))
    connection.execute(dcreate)
    connection.execute(text('COMMIT;'))
    
    # Add record for each datum in dataset
    dinsert = []
    dupdate = []
    for data in config['Data']:
        
        # Get metadata
        dschema = data.get('Upload Schema', 'public')
        dname = data['Name'] + f'_v{ver}'
        dtitle = data.get('Title', '')
        dformat = 'csv' if 'Geometry Column' not in data else 'geojson'
        dspatial = 'Geometry Column' in data
        dpermission = data.get('Permission', 'user')
        dpublish = str(datetime.now().astimezone())
        dtag = data.get('Tag', '')
        ddesc = data.get('Description', '').replace("'", "''")
        dcat = data.get('Category', '')
        dcontact = config.get('Contact', 'support@openmortality.org')

        # Add metadata to desc
        dmeta = data.get('Metadata', None)
        if isinstance(dmeta, dict):
            dmeta_list = '\n'.join([f'* **{k}**: {v}' for k, v in dmeta.items()])
            ddesc = ddesc + '\n' + dmeta_list
        
        # Add file, rows and cols to desc
        cols = ddescribe[ddescribe['dataset'] == data['Name']]['columns'].tolist()[0]
        rows = ddescribe[ddescribe['dataset'] == data['Name']]['rows'].tolist()[0]
        ddesc = ddesc + f'\n* **File**: {dname}.{dformat}\n* **Dimensions**: {"{:,}".format(cols)} columns / {"{:,}".format(rows)} rows'

        # Add data record insert sql
        dinsert.append(dedent(
            f"""
            --- {dname} data record
            INSERT INTO {om_dschema}.{om_dtable} (title, data_name, \"format\", is_spatial, \"permission\", publish_date, status, tag, data_desc, category, contact, data_columns, data_rows)
            SELECT '{dtitle}', '{dname}', '{dformat}', '{dspatial}', '{dpermission}', '{dpublish}', 'published', '{dtag}', '{ddesc}', '{dcat}', '{dcontact}', '{cols}', '{rows}'
            WHERE NOT EXISTS (SELECT 1 FROM {om_dschema}.{om_dtable} WHERE data_name = '{dname}');
            """
        ))

        # Update data record
        dupdate.append(dedent(
            f"""
            --- {dname} dataset record
            UPDATE {om_dschema}.{om_dtable}
            SET
                title = '{dtitle}',
                data_name = '{dname}',
                \"format\" = '{dformat}',
                is_spatial = '{dspatial}',
                \"permission\" = '{dpermission}',
                last_updated_date = '{dpublish}',
                status = 'published',
                tag = '{dtag}',
                data_desc = '{ddesc}',
                category = '{dcat}',
                contact = '{dcontact}',
                data_columns = '{cols}',
                data_rows = '{rows}' 
            WHERE data_name = '{dname}';
            """
        ))
        
    # Add data record
    dinsert = ''.join(dinsert)
    dupdate = ''.join(dupdate)
    connection.execute(text(dinsert))
    connection.execute(text(dupdate))

## Add Dataset Record

Query for the current dataset table or create one if it does not exist.

Then add a record for this dataset.

In [103]:
om_dstable = config['Dataset Table']
om_dsschema = config.get('Dataset Schema', 'public')

with uengine.connect() as connection:
    
    # Create dataset table if not exists
    dscreate = text(dedent(
        f"""
        --- Dataset records table
        CREATE TABLE IF NOT EXISTS {om_dsschema}.{om_dstable} (
            dataset_id SERIAL PRIMARY KEY,
            title VARCHAR UNIQUE,
            title_abbr VARCHAR UNIQUE,
            status VARCHAR,
            data_id VARCHAR,
            publish_date TIMESTAMP WITH TIME ZONE,
            tag VARCHAR,
            dataset_desc VARCHAR,
            contact VARCHAR,
            last_update_date TIMESTAMP WITH TIME ZONE,
            permission VARCHAR
        );
        """
    ))
    connection.execute(dscreate)
    connection.execute(text('COMMIT;'))
    
    # Get data ids
    dquery = text(
        f"""
        SELECT data_id, data_name FROM {om_dschema}.{om_dtable} WHERE data_name = ANY(:names);
        """
    )
    dids = pd.read_sql(dquery, uengine, params={
        'names': [d['Name'] + f'_v{ver}' for d in config['Data']]
    })
    dids = ','.join(dids['data_id'].astype(str).tolist())
    
    # Get markdown description
    with open(f'../README.md', 'r') as file:
        md = file.read()
    ddesc = re.split(r'(\n#)', md) # split by sections
    ddesc = [ddesc[i-1].strip() + d if i > 0 and '\n#' == ddesc[i -1] else d for i, d in enumerate(ddesc)] # join back hashtags
    ddesc = [d for d in ddesc if '## About' in d or '## Citation' in d] # filter for certain sections
    ddesc = '\n\n'.join(ddesc)
    ddesc = ddesc.replace('## About\n\n', '') # Remove about section title
    ddesc = ddesc.replace("'", "''") # replace single quotes with double for pg
    
    # Set metadata
    dstitle = config['Title']
    dstitle_abbr_default = ''.join(filter(str.isupper, dstitle.title())).lower()
    dstitle_abbr = config.get('Abbreviation', dstitle_abbr_default)
    dscontact = config.get('Contact', 'support@openmortality.org')
    dspublish = str(datetime.now().astimezone())
    dstag = config.get('Tag', '')
    dspermission = config.get('Permission', 'user')
    
    # Add dataset record
    dsinsert = text(dedent(
        f"""
        --- {dstitle_abbr} dataset record
        INSERT INTO {om_dsschema}.{om_dstable} (title, title_abbr, status, data_id, contact, publish_date, tag, dataset_desc, \"permission\")
        SELECT '{dstitle}', '{dstitle_abbr}', 'published', '{dids}', '{dscontact}', '{dspublish}', '{dstag}', '{ddesc}', '{dspermission}'
        WHERE NOT EXISTS (SELECT 1 FROM {om_dsschema}.{om_dstable} WHERE title = '{dstitle}');
        """
    ))
    connection.execute(dsinsert)

    # Update dataset record
    dsupdate = text(dedent(
        f"""
        --- {dstitle_abbr} dataset record
        UPDATE {om_dsschema}.{om_dstable}
        SET
            title = '{dstitle}',
            title_abbr = '{dstitle_abbr}',
            status = 'published',
            data_id = '{dids}',
            contact = '{dscontact}',
            publish_date = '{dspublish}',
            tag = '{dstag}',
            dataset_desc = '{ddesc}',
            \"permission\" = '{dspermission}'
        WHERE title = '{dstitle}';
        """
    ))
    connection.execute(dsupdate)

## Grant Access to User

Grant access to OM user.

In [104]:
if is_om:
    omuser = config['Database User']
    with uengine.connect() as connection:
        
        # Create grant sql
        grantsql = ['--- Grant select on data for OM user']
        for data in config['Data']:
            dtable = data['Name'] + f'_v{ver}'
            dschema = data.get('Upload Schema', 'public')
            grantsql.append(f'GRANT SELECT ON TABLE {dschema}.{dtable} TO {omuser};')
        grantsql = text('\n'.join(grantsql))
        
        # Grant access to user for added tables
        connection.execute(grantsql)
        connection.execute(text('COMMIT;'))

## Refresh Dataset Records View

Refresh materialized view for dataset records.

In [105]:
if is_om:
    matview = config['Dataset Refresh View']
    with uengine.connect() as connection:
        refreshsql = text(f'--- Refresh dataset records materialized view \nREFRESH MATERIALIZED VIEW {matview};')
        connection.execute(refreshsql)
        connection.execute(text('COMMIT;'))

## Save Data Records SQL

Save data records sql to file.

In [106]:
with open(f'database/{abbrv}_data_v{ver}.sql', 'w') as file:
    if is_om:
        file.write(str(dcreate) + str(dupdate) + str(dinsert) + '\n' + str(grantsql))
    else:
        file.write(str(dcreate) + str(dupdate) + str(dinsert))

## Save Dataset Records SQL

Save dataset records sql to file.

In [107]:
with open(f'database/{abbrv}_dataset_v{ver}.sql', 'w') as file:
    if is_om:
        file.write(str(dscreate) + str(dsupdate) + str(dsinsert) + '\n' + str(refreshsql))
    else:
        file.write(str(dscreate) + str(dsupdate) + str(dsinsert) + '\n')

# Close Database

In [108]:
engine.dispose()
uengine.dispose()